In [ ]:
%%javascript
document.title="numerics"

In [ ]:
import os
from collections import namedtuple
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import axes3d

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})

In [ ]:
def load_sln(version, path, file_names):
    sln = Sln._make([pd.read_csv(os.path.join(path, '{0}_{1}.csv'.format(file, version)), 
                     header=None) for file in file_names])
    return sln 

def plot_psi(psi):
    x = np.arange(len(psi))*360/J
    ax = sns.lineplot(data=psi.set_axis(x))
    ax.set_xlabel('α')
    ax.set_ylabel('$\psi$')
    return ax

In [ ]:
path = '/Users/kevinliu/temp/CarNumericsData/'
data_files = ['psi','pressure','h','theta_A', 'theta_B']

Sln = namedtuple("Sln", data_files)

v = 0
versions = range(v,v+3)
slns = [load_sln(v, path, data_files) for v in versions]

I, J = slns[0].psi.shape
dr = 1/I
da = (2*np.pi)/J

## asymptotics

In [ ]:
# asymptotics
q = 1
theta_1 = 0

r = np.reshape(np.arange(dr, 1+dr, dr), (-1,1))

alpha = np.reshape(np.arange(1,J+1)*2*np.pi/J, (-1,1))
alpha_minus = alpha[int(J/4):int(3*J/4)]
alpha_plus = np.concatenate((alpha[:int(J/4)], alpha[int(3*J/4):]))

x_minus = np.dot(r, np.cos(alpha_minus).transpose())
x_plus = np.dot(r, np.cos(alpha_plus).transpose())
z_minus = q*np.dot(r, np.sin(alpha_minus).transpose())
z_plus = q*np.dot(r, np.sin(alpha_plus).transpose())

f_x = theta_1 + 2*x_minus
f_xx = 2

#f_x = theta_1
#f_xx = 0

a_p_minus = 0.5*(z_minus**2 - (q**2)*(1-x_minus**2))*f_xx + (q**2)*x_minus*f_x
a_p_plus = 0.5*(z_plus**2 - (q**2)*(1-x_plus**2))*f_xx
a_p = np.concatenate((a_p_plus[:,:int(J/4)], a_p_minus, a_p_plus[:,int(J/4):]),axis=1)

In [ ]:
i = 2
fig = plt.figure()
p = slns[0].pressure
ax = p.iloc[i].plot()
ax = pd.DataFrame(a_p).iloc[i].plot()
ax.legend(['Numerics','Asymptotic'])

## numerical plots

In [ ]:
angles = (40, 180)

pressure = slns[1].psi

fig = plt.figure()
ax = plt.axes(projection="3d")
q = 0.1
x = np.arange(-1, 1+0.01, 0.01)
y = q*np.arange(-1, 1+0.01, 0.01)
x,y = np.meshgrid(x, y)

N = 4
#z = np.cos(0.5*np.pi*x)*np.cos(0.5*np.pi*y/q)
z = x*np.cos(2*np.pi*y*N/q)
#np.cos(2*np.pi*x/N)*np.cos(2*np.pi*y*N/q)
#np.cos(0.5*np.pi*x)*np.cos(4*np.pi*y/q)

ax.plot_surface(x,y,z)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')

ax.view_init(*angles)
plt.show()


fig = plt.figure()
ax = plt.axes(projection="3d")

r = np.arange(1,I+1)/I
alpha = np.arange(1,J+1)*2*np.pi/J
alpha, r = np.meshgrid(alpha, r, indexing='xy')

x = r*np.cos(alpha)
y = q*r*np.sin(alpha)
z = pressure.values.reshape(I,-1)

ax.plot_surface(x,y,z)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')

ax.view_init(*angles)
plt.show()


fig = plt.figure()
ax = plt.axes(projection="3d")

ax.plot_surface(alpha,r,z)
ax.set_xlabel('alpha')
ax.set_ylabel('r')
ax.set_zlabel('z')

ax.view_init(*angles)
plt.show()

In [ ]:
plt.plot(z[:,99])

x = np.arange(-1,1,0.01)
y = np.cos(0.5*x*np.pi)

#g(x) = -z*cos(2pi*x)

df = pd.DataFrame({'x':x,'y':y})
fig = plt.figure()
df.plot(x='x')
plt.title('g(x)')

In [ ]:
i = 99
t = 50

labels = ['x_c = {}'.format(i) for i in [-1,0,1]]

fig = plt.figure()
for p in [slns[v].psi for v in range(len(slns))]:
    c_p = p.iloc[i]
    c_p.index = np.arange(1,J+1)*360/J
    ax = c_p.plot()
    #pp = pd.DataFrame({'Pressure':p.iloc[i], 'alpha':np.arange(1,J+1)*360/J})
    #ax = pp.plot(x='alpha')
ax.legend(labels)
plt.title('Pressure at r={0}*dr, t={1}*dt'.format(i+1, t))

fig = plt.figure()
for h in [slns[v].h[:t] for v in range(len(slns))]:
    ax = h.iloc[:,0].plot()
ax.legend(labels)
plt.title('h at t={}*dt'.format(t))

fig = plt.figure()
for theta in [slns[v].theta_A[:t] for v in range(len(slns))]:
    ax = theta.iloc[:,0].plot()
ax.legend(labels)
plt.title('theta1 at t={}*dt'.format(t))

fig = plt.figure()
for theta in [slns[v].theta_B[:t] for v in range(len(slns))]:
    ax = theta.iloc[:,0].plot()
ax.legend(labels)
plt.title('theta2 at t={}*dt'.format(t))


## plot slns

In [ ]:
def df_dx(idx, J, dr, da, theta1, theta2):
    alpha = np.arange(da, 2*np.pi+da, da)
    r = (idx+1)*dr;

    df_dr = np.cos(alpha)*theta1 + np.sin(alpha)*theta2;
    df_da = r*(np.cos(alpha)*theta2 - np.sin(alpha)*theta1);
    
    res = df_dr*np.cos(alpha)-df_da*np.sin(alpha)/r 
        #-3*(np.sin(alpha) + np.cos(alpha))
        #df_dr*np.cos(alpha)-df_da*np.sin(alpha)/r 
    return res


def check_sln(psi, idx, J, dr, da, theta1, theta2, rhs_func):
    c_psi = psi.iloc[idx]
    r = (idx+1)*dr
    alpha = np.arange(da, 2*np.pi+da, da)

    psi_2 = c_psi.values
    s1_psi_2 = c_psi.shift(1).fillna(c_psi.values[-1]).values
    s2_psi_2 = c_psi.shift(-1).fillna(c_psi.values[0]).values
    
    psi_1 = psi.iloc[idx-1].values
    psi_3 = psi.iloc[idx+1].values
    
    d2psi_dr2 = (psi_3+psi_1-2*psi_2)/(dr**2)
    dpsi_dr = (psi_3 - psi_1)/(2*dr)
    d2psi_da2 = (s1_psi_2+s2_psi_2-2*psi_2)/(da**2)
    
    num_soln = d2psi_dr2 + dpsi_dr/r + d2psi_da2/np.power(r,2)
    real_soln = -rhs_func(idx, J, dr, da, theta1, theta2)

    return num_soln, real_soln

In [ ]:
num, rel = check_sln(psi, 2, J, dr, da, -1, 0, df_dx)
pd.DataFrame({'num':num}).plot()

In [ ]:
pressure = pressures[2]
fig = plt.figure()
ax = plt.axes(projection="3d")

alpha = np.arange(1,J+1)*360/J
r = np.arange(1,I+1)/I
alpha, r = np.meshgrid(alpha, r)

z = pressure.values.reshape(I,-1)

ax.plot_surface(alpha,r,z)
ax.set_xlabel('α')
ax.set_ylabel('r')
ax.set_zlabel('z')

ax.view_init(20, 75)

plt.show()

In [ ]:
hs[2].plot()

# circle numerics

In [ ]:
pressure.iloc[99].plot()

In [ ]:
fig = plt.figure()
ax = plt.axes(projection="3d")

alpha = np.arange(1,J+1)*360/J
r = np.arange(1,I+1)/I
alpha, r = np.meshgrid(alpha, r)

z = pressure.values.reshape(I,-1)

ax.plot_surface(alpha,r,z)
ax.set_xlabel('α')
ax.set_ylabel('r')
ax.set_zlabel('z')

ax.view_init(20, 75)

plt.show()

In [ ]:
h.plot()

In [ ]:
theta1.plot()

## test solution

In [ ]:
num, rel = check_sln(psi, 1, J, dr, da, 1, 0, df_dx)
pd.DataFrame({'num':num,'rel':rel}).plot()
pd.Series(num-rel).describe()

In [ ]:
num, rel = check_sln(psi, 98, J, dr, da, 1, 0, df_dx)
pd.DataFrame({'num':num,'rel':rel}).plot()
pd.Series(num-rel).describe()

In [ ]:
i = 98
r = (i+1)*dr
alphas = (np.arange(J)+1)*2*np.pi/J
real_sln = r**2*(np.sin(alphas) + np.cos(alphas))

In [ ]:
pd.DataFrame({'real': real_sln,'num':psi.iloc[i]}).plot()

In [ ]:
pd.Series(real_sln - psi.iloc[i]).plot()

## v1

In [ ]:
i = I
plot_ids = [i] #1*d_r, 100*d_r = R = 1
for j in plot_ids:
    c_psi = get_psi(psi, j)
    ax = plot_psi(c_psi) 
ax.legend(plot_ids)

ans = pd.DataFrame({'α':np.arange(len(c_psi))*360/J, 
                    'Ψ':[np.power((i*dr),2)*(np.sin(x*da) + np.cos(x*da)) for x in range(1,J+1)]})
ans.plot(x='α',y='Ψ', ax = ax)
ax.legend(['soln r={}*dr'.format(i),'actual r={}*dr'.format(i)])

errs.describe()

In [ ]:
j=2000
i = I
plot_ids = [i] #1*d_r, 100*d_r = R = 1
for j in plot_ids:
    c_psi = get_psi(psi, j)
    ax = plot_psi(c_psi) 
ax.legend(plot_ids)

ans = pd.DataFrame({'α':np.arange(len(c_psi))*360/J, 
                    'Ψ':[np.power((i*dr),2)*(np.sin(x*da) + np.cos(x*da)) for x in range(1,J+1)]})
ans.plot(x='α',y='Ψ', ax = ax)
ax.legend(['soln r={}*dr'.format(i),'actual r={}*dr'.format(i)])

errs.describe()

In [ ]:
i = I
plot_ids = [i] #1*d_r, 100*d_r = R = 1
for j in plot_ids:
    c_psi = get_psi(psi, j)
    ax = plot_psi(c_psi) 
ax.legend(plot_ids)

ans = pd.DataFrame({'α':np.arange(len(c_psi))*360/J, 
                    'Ψ':[np.power((i*dr),2)*(np.sin(x*da) + np.cos(x*da)) for x in range(1,J+1)]})
ans.plot(x='α',y='Ψ', ax = ax)
ax.legend(['soln r={}*dr'.format(i),'actual r={}*dr'.format(i)])

errs.describe()

In [ ]:
j=2000
i= I
plot_ids = [i] #1*d_r, 100*d_r = R = 1
for j in plot_ids:
    c_psi = get_psi(psi, j)
    ax = plot_psi(c_psi) 
ax.legend(plot_ids)

ans = pd.DataFrame({'α':np.arange(len(c_psi))*360/J, 
                    'Ψ':[np.power((i*dr),2)*(np.sin(x*da) + np.cos(x*da)) for x in range(1,J+1)]})
ans.plot(x='α',y='Ψ', ax = ax)
ax.legend(['soln r={}*dr'.format(i),'actual r={}*dr'.format(i)])

errs.describe()

In [ ]:
i = 1000
plot_ids = [i] #1*d_r, 100*d_r = R = 1
for j in plot_ids:
    c_psi = get_psi(psi, j)
    ax = plot_psi(c_psi) 
ax.legend(plot_ids)

ans = pd.DataFrame({'α':np.arange(len(c_psi))*360/J, 
                    'Ψ':[np.power((i*dr),2)*(np.sin(x*da) + np.cos(x*da)) for x in range(1,J+1)]})
ans.plot(x='α',y='Ψ', ax = ax)
ax.legend(['soln r={}*dr'.format(i),'actual r={}*dr'.format(i)])

errs.describe()

In [ ]:
i = 1000
j = 2000
plot_ids = [i] #1*d_r, 100*d_r = R = 1
for j in plot_ids:
    c_psi = get_psi(psi, j)
    ax = plot_psi(c_psi) 
ax.legend(plot_ids)

ans = pd.DataFrame({'α':np.arange(len(c_psi))*360/J, 
                    'Ψ':[np.power((i*dr),2)*(np.sin(x*da) + np.cos(x*da)) for x in range(1,J+1)]})
ans.plot(x='α',y='Ψ', ax = ax)
ax.legend(['soln r={}*dr'.format(i),'actual r={}*dr'.format(i)])

errs.describe()

i = 1
plot_ids = [i] #1*d_r, 100*d_r = R = 1
for j in plot_ids:
    c_psi = get_psi(psi, j)
    ax = plot_psi(c_psi) 
ax.legend(plot_ids)

ans = pd.DataFrame({'x':np.arange(len(c_psi))*360/J, 
                    'psi':[np.power((i*dr),2)*(np.sin(x*da) + np.cos(x*da)) for x in range(1,J+1)]})
ans.plot(x='x',y='psi', ax = ax)
ax.legend(['soln','actual'])

In [ ]:
np.power(2*0.1, -2)

## plot errors

In [ ]:
ax = errs.plot()

ax.set_title('Mean absolute error (numerical v.s. actual) for r in R')

In [ ]:
ax = errs.plot()

ax.set_title('Mean absolute error (numerical v.s. actual) for r in [1 ... 99]*dr')

# 4th order accuracy

In [ ]:
errs.describe()

In [ ]:
ax = errs.plot()

ax.set_title('Mean absolute error (numerical v.s. actual) for r in [1 ... 99]*dr')

pd.Series ([np.power((i*dr),2)*(np.sin(x*da) + np.cos(x*da)) for x in range(1,J+1)]).plot()

In [ ]:
import json
import yfinance as yf
from datetime import datetime

dt = datetime.now()

datetime.strftime(dt, '%Y-%m-%d')

data = yf.download("BTC-USD", start="2020-01-01", end="2021-04-30")

data.reset_index()